### Import Required Libraries and Set Up Environment Variables

In [32]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
pd.set_option('display.max_columns', 30)

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv('.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}&api-key={nyt_api_key}"

In [4]:
# verify that response code is not an error, looking for <response [200]>
response_data = requests.get(query_url)
print(response_data)

<Response [200]>


In [5]:
# Test to see which API keys are being used
# print(nyt_api_key)
# print(tmdb_api_key)

In [6]:
# See json data
response_data.json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html',
    'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.',
    'source': 'The New York Times',
    'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick',
     'kicker': None,
     'content_kicker': None,
     'print_headline': 'The Attachment Diaries',
     'name': None,
     'seo': None,
     'sub': None},
    'keywords': [{'name': 'subject',
      'value': 'Movies',
      'rank': 1,
      'major': 'N'},
     {'name': 'creative_works',
      'value': 'The Attachment Diaries (Movie)',
      'rank': 2,
      'major': 'N'},
     {'name': 'persons',
      'value': 'Diment, Valentin Javier',
      'rank': 3,
      'major': 'N'}],
    'pub_date': '2023-05-25T11:00:03+0000',
    'byline': {'orig

In [7]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    
    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}&api-key={nyt_api_key}"
    query_url = f"{query_url}&page={str(page)}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page {page} had no results")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [30]:
# See first five resulting headlines
for review in reviews_list[:5]:
    print(review["headline"])

{'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}
{'main': 'Review: ‘What’s Love Got to Do With It?’ Probably a Lot', 'kicker': None, 'content_kicker': None, 'print_headline': 'What’s Love Got to Do With It?', 'name': None, 'seo': None, 'sub': None}
{'main': '‘You Can Live Forever’ Review: Do You Love Me Now?', 'kicker': None, 'content_kicker': None, 'print_headline': 'You Can Live Forever', 'name': None, 'seo': None, 'sub': None}
{'main': '‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip', 'kicker': None, 'content_kicker': None, 'print_headline': 'A Tourist’s  Guide to Love', 'name': None, 'seo': None, 'sub': None}
{'main': '‘Other People’s Children’ Review: True Romance', 'kicker': 'Critic’s pick', 'content_kicker': None, 'print_headline': 'Intoxicating Love With a Sobering Turn', 'name': None, 'seo': None, 'sub': None}


In [9]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
# can't use .head() in this instance, because .head() is a panda function that works with dataframes,
# but this data is not in a dataframe, it's in a list. Also, json.dumps does not work on DataFrame objects
first_five = reviews_list[:5]
print(json.dumps(first_five,indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [10]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)
reviews_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [11]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# This is the code to accomplish extracting title using unicode characters as suggested by instructions.
reviews_list_df["title"] = reviews_list_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review:")])

# This is the code to accomplish extracting title using single quotations with an escape character instead of using the unicode values.
# reviews_list_df["title"] = reviews_list_df["headline.main"].apply(lambda st: st[st.find("‘")+1:st.find("’ Review")])
reviews_list_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [12]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

In [13]:
# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df_new = reviews_list_df
reviews_list_df_new['keywords'] = reviews_list_df['keywords'].apply(extract_keywords)
reviews_list_df_new.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [14]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

titles = reviews_list_df_new["title"]
titles_list = titles.to_list()
titles_list




['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [15]:
# Prepare The Movie Database query
url2 = "https://api.themoviedb.org/3/search/movie?query="
url3 = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key


In [31]:
# Test to see which API keys are being used
# print(nyt_api_key)
# print(tmdb_api_key)

In [17]:
# test query with one example to be sure working
title_ex = "A Love Song"
query_url_2 = f"{url2}{title_ex}{tmdb_key_string}"
test = requests.get(query_url_2).json()

print(json.dumps(test,indent=4))

{
    "page": 1,
    "results": [
        {
            "adult": false,
            "backdrop_path": "/5CLcdmZJgYpwF46n7rtgWpSIQcz.jpg",
            "genre_ids": [
                18,
                10749
            ],
            "id": 913851,
            "original_language": "en",
            "original_title": "A Love Song",
            "overview": "Two childhood sweethearts, now both widowed, share a night by a lake in the mountains.",
            "popularity": 7.827,
            "poster_path": "/uu3FvDIV9Bw7I2owLUR44IBLH0D.jpg",
            "release_date": "2022-07-29",
            "title": "A Love Song",
            "video": false,
            "vote_average": 7.05,
            "vote_count": 30
        },
        {
            "adult": false,
            "backdrop_path": null,
            "genre_ids": [
                10749,
                18
            ],
            "id": 269579,
            "original_language": "es",
            "original_title": "Un cuento de circo & a lov

In [18]:

# Create an empty list to store the results
tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests

request_counter = 1
max_requests = 50

for title in titles_list:
# Loop through the titles

    # Check if we need to sleep before making a request
    if request_counter % max_requests == 0:
        time.sleep(1)
        print(f"application is sleeping for 1 second after {request_counter} requests")
    
    # Include a try clause to search for the full movie details.    
    try:
        
        # Perform a "GET" request for The Movie Database
        movie_id = []
        query_url_2 = f"{url2}{title}{tmdb_key_string}"
        movie_response = requests.get(query_url_2).json()
        movie_id.append(movie_response['results'][0]['id'])
        
        # Make a request for a the full movie details
        for id in movie_id:
            
            # Execute "GET" request with url
            query_url_3 = f"{url3}{id}?api_key={tmdb_api_key}"
            movie_response_all = requests.get(query_url_3).json()
            
            # Extract the genre names into a list
            genres = []
            for genre in movie_response_all["genres"]:    
                genres.append(genre['name'])
            
            # Extract the spoken_languages' English name into a list
            spoken_languages = []
            for spoken_lang in movie_response_all["spoken_languages"]:
                spoken_languages.append(spoken_lang['english_name'])
            
            # Extract the production_countries' name into a list
            production_countries = []
            for country in movie_response_all["production_countries"]:
                production_countries.append(country['name'])
            
            # Add the relevant data to a dictionary and
            results_dict = {
                'title': movie_response_all['title'],
                'original_title': movie_response_all['original_title'],
                'budget': movie_response_all['budget'],
                'original_language': movie_response_all['original_language'],
                'homepage': movie_response_all['homepage'],
                'overview': movie_response_all['overview'],
                'popularity': movie_response_all['popularity'],
                'runtime': movie_response_all['runtime'],
                'revenue': movie_response_all['revenue'],
                'release_date': movie_response_all['release_date'],
                'vote_average': movie_response_all['vote_average'],
                'vote_count': movie_response_all['vote_count'],
                'genres': genres,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries
            }
            
            # append the results_dict to the tmdb_movies_list list
            tmdb_movies_list.append(results_dict)
            
            # print out the title that was found
            print(f"found: {title}")
    
    # Use the except clause to print out a statement if a movie
    # is not found.        
    except Exception as e:
        print(f"not found: {title}")
    
    # Add 1 to the request counter
    request_counter += 1
    

        

found: The Attachment Diaries
not found: What’s Love Got to Do With It?’ Probably a Lo
found: You Can Live Forever
found: A Tourist’s Guide to Love
found: Other People’s Children
found: One True Loves
found: The Lost Weekend: A Love Story
found: A Thousand and One
found: Your Place or Mine
found: Love in the Time of Fentanyl
found: Pamela, a Love Story
found: In From the Side
found: After Love
found: Alcarràs
found: Nelly & Nadine
found: Lady Chatterley’s Lover
found: The Sound of Christmas
found: The Inspection
found: Bones and All
found: My Policeman
found: About Fate
found: Waiting for Bojangles
found: I Love My Dad
found: A Love Song
found: Alone Together
found: Art of Love
found: The Wheel
found: Thor: Love and Thunder
found: Both Sides of the Blade
found: Fire of Love
found: Love & Gelato
found: Stay Prayed Up
found: Benediction
found: Dinner in America
found: In a New York Minute
found: Anaïs in Love
found: I Love America
found: See You Then
found: La Mami
found: Love After Love

In [19]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

first_5_tmdb = tmdb_movies_list[:5]
print(json.dumps(first_5_tmdb,indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.522,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [20]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

In [21]:
# Preview first 5 rows of dataframe
tmdb_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.522,102,0,2021-10-07,4.000,2,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",7.276,96,0,2022-06-11,7.263,38,"[Drama, Romance]","[French, English]",[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",11.828,96,0,2023-04-21,6.388,138,"[Romance, Comedy, Adventure]",[English],[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",7.946,104,0,2022-09-21,6.917,156,[Drama],[French],[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,11.423,100,37820,2023-04-07,6.761,47,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]"


### Merge and Clean the Data for Export

In [22]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,reviews_list_df_new, on=["title"])

# Preview merge
merged_df.head()


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.522,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",7.276,96,0,2022-06-11,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,11.423,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.899,95,0,2023-04-13,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.664,116,0,2023-03-31,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [33]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres","spoken_languages","production_countries"]

# Create a list of characters to remove
characters_to_remove = ["'","[","]"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char,'')

# Display the fixed DataFrame
merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries,...,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.522,102,0,2021-10-07,4.000,2,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",7.276,96,0,2022-06-11,7.263,38,"Drama, Romance","French, English",Canada,...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,11.423,100,37820,2023-04-07,6.761,47,"Romance, Drama, Comedy",English,"Germany, United States of America",...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.899,95,0,2023-04-13,6.000,2,Documentary,English,United States of America,...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.664,116,0,2023-03-31,7.020,49,Drama,English,United States of America,...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [24]:
# Get list of columns to see byline.person column
merged_df.columns


Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [25]:
# Drop "byline.person" column
merged_df_new = merged_df.drop('byline.person', axis=1)

# Get list of columns in new merged dataframe to confirm byline.person column has been removed
merged_df_new.columns

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original',
       'byline.organization'],
      dtype='object')

In [26]:
# Get dataframe row count
len(merged_df_new)

122

In [27]:
# Delete duplicate rows and reset index and recheck row count
merged_df_new_reset = merged_df_new.drop_duplicates().reset_index(drop=True)
# Get row count after duplicate drop
len(merged_df_new_reset)

122

In [34]:
# Check reset index
merged_df_new_reset.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.522,102,0,2021-10-07,4.000,2,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",7.276,96,0,2022-06-11,7.263,38,"Drama, Romance","French, English",Canada,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,11.423,100,37820,2023-04-07,6.761,47,"Romance, Drama, Comedy",English,"Germany, United States of America",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.899,95,0,2023-04-13,6.000,2,Documentary,English,United States of America,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.664,116,0,2023-03-31,7.020,49,Drama,English,United States of America,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [29]:
# Export data to CSV without the index
merged_df_new_reset.to_csv("Outputs/movie_data.csv", index=False)